# 1. 引入package

In [17]:
import tensorflow as tf
import keras

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import copy

In [2]:
#from ckiptagger import WS, POS, NER

In [3]:
def word_proc_dataframe(input_Series,
              column_name = 'Text',
              tt_seg = (1,10),
              export_flag = False, 
              #tt_begin = 0,tt_end = 50,
              output_file = "df_test_out.csv"):
    """
        data_path(str):資料路徑(字串)
        column_name(str):要處理的欄位名稱
        tt_seg(tuple):起訖列數
        export_flag(boolean):是否產出檔案
        output_file:輸出的檔案名稱，指定為CSV檔    
    """
    from ckiptagger import WS, POS, NER
    
    """
    file_type = data_path[-4:]
    print("輸入檔案格式為 {}".format(file_type))
    if 'csv' in file_type:
        data = pd.read_csv(data_path)
    elif 'xls' in file_type:
        data = pd.read_excel(data_path)
    else:
        #print("檔案須為CSV或XLS/XLSX")
        raise Exception("檔案須為CSV或XLS/XLSX")
        #return None
    """
    
    """
    column_list = list(input_df.columns)
    if column_name in column_list:
        text = input_df[column_name]
    else:
        print("輸入欄位\"{input_column}\"不存在\n匯入檔案欄位{i_column_name}".format(input_column = column_name,
                                                                       i_column_name=str(column_list)))
        raise Exception("不存在此欄位名稱")
    """
    
    #ls_Test = list(text[tt_begin:tt_end])
    
    ###########      debug        ###############
    print("起始列數：{begin}\n結束列數:{end}".format(begin = (tt_seg[0]),end = tt_seg[1]))
    print("列印前5筆{}".format(input_Series[:5]))
    #print(input_Series[:10])
    #print(input_Series[tt_seg[0]:tt_seg[1]])
    ###########      debug        ###############
    ls_len = len(input_Series)
    

    ls_Test = list(input_Series[:])
    
    
    if (len(ls_Test) > 0):
        print("取得list，長度為：{list_len}\n 載入 WS 資料...".format(list_len = len(ls_Test)))
    else:
        raise Exception("資料長度為0")
        
        
    
    ws = WS("./data")
    print("============================\n 開始執行 WS\n============================\n")
    word_sentence_list = ws(ls_Test
                                # sentence_segmentation=True, # To consider delimiters
                                # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}),
                                # This is the defualt set of delimiters
                                # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
                                # coerce_dictionary = dictionary2, # words in this dictionary are forced
                            )
    del ws
    del WS
    gc.collect()
    print("============================\n WS 完成\n============================\n載入 POS 資料...")
    
    print("============================\n 開始執行 POS \n============================\n")
    pos = POS("./data")
    pos_sentence_list = pos(word_sentence_list)
    del pos
    del POS
    gc.collect()
    print("============================\n POS 完成\n============================\n載入NER 資料...")
    
    
    print("============================\n 開始執行 NER \n============================\n")
    ner = NER("./data")
    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
    del NER
    del ner
    gc.collect()
    print("============================\n NER 完成\n============================\n")
        
    df_Test = pd.DataFrame(ls_Test,columns=[column_name])
    df_Test['WS'] = np.array(word_sentence_list)
    
    ##################
    #print("DEBUG")
    #print(pos_sentence_list[:5])
    ##################
    
    df_Test['POS'] = np.array(pos_sentence_list)
    df_Test['NER'] = np.array(entity_sentence_list)
    print("============================\n Dataframe Created \n============================\n")
    
    if export_flag is True:
        df_Test.to_csv(output_file)
        print("File Exported!!!!!")
    else:
        print("斷詞處理完成，不產生檔案")
        
    #return df_Test,word_sentence_list,pos_sentence_list,entity_sentence_list
    return entity_sentence_list


In [4]:
def word_proc_path_all(data_path,
              column_name,
              #tt_seg = (1,10),
              export_flag = False, 
              #tt_begin = 0,tt_end = 50,
              output_file = "df_test_out.csv"):
    """
        ===============================
        108.12.10 改為不用輸入起訖列數
        ===============================
        
        
        data_path(str):資料路徑(字串)
        column_name(str):要處理的欄位名稱
        tt_seg(tuple):起訖列數
        export_flag(boolean):是否產出檔案
        output_file:輸出的檔案名稱，指定為CSV檔    
    """
    from ckiptagger import WS, POS, NER
    
    file_type = data_path[-4:]
    print("輸入檔案格式為 {}".format(file_type))
    if 'csv' in file_type:
        data = pd.read_csv(data_path)
    elif 'xls' in file_type:
        data = pd.read_excel(data_path)
    else:
        #print("檔案須為CSV或XLS/XLSX")
        raise Exception("檔案須為CSV或XLS/XLSX")
        #return None
    
    column_list = list(data.columns)
    if column_name in column_list:
        text = data[column_name]
    else:
        print("輸入欄位\"{input_column}\"不存在\n匯入檔案欄位{i_column_name}".format(input_column = column_name,
                                                                       i_column_name=str(column_list)))
        raise Exception("不存在此欄位名稱")

    
    #ls_Test = list(text[tt_begin:tt_end])
    #print("起始列數：{begin}\n結束列數:{end}".format(begin = (tt_seg[0]-1),end = tt_seg[1]))
    #ls_Test = list(text[(tt_seg[0]-1):tt_seg[1]])
    ls_Test = list(text)
    
    if (len(ls_Test) > 0):
        print("取得list，長度為：{list_len}\n 載入 WS 資料...".format(list_len = len(ls_Test)))
    else:
        raise Exception("資料長度為0")
    
    
    print("Got list\n 載入 WS 資料...")
    ws = WS("./data")
    print("============================\n 開始執行 WS\n============================\n")
    word_sentence_list = ws(ls_Test
                                # sentence_segmentation=True, # To consider delimiters
                                # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}),
                                # This is the defualt set of delimiters
                                # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
                                # coerce_dictionary = dictionary2, # words in this dictionary are forced
                            )
    del ws
    del WS
    gc.collect()
    print("============================\n WS 完成\n============================\n載入 POS 資料...")
    
    print("============================\n 開始執行 POS \n============================\n")
    pos = POS("./data")
    pos_sentence_list = pos(word_sentence_list)
    del pos
    del POS
    gc.collect()
    print("============================\n POS 完成\n============================\n載入NER 資料...")
    
    
    print("============================\n 開始執行 NER \n============================\n")
    ner = NER("./data")
    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
    del NER
    del ner
    gc.collect()
    print("============================\n NER 完成\n============================\n")
        
    df_Test = pd.DataFrame(ls_Test,columns=[column_name])
    df_Test['WS'] = np.array(word_sentence_list)
    
    ##################
    #print("DEBUG")
    #print(pos_sentence_list[:5])
    ##################
    
    df_Test['POS'] = np.array(pos_sentence_list)
    df_Test['NER'] = np.array(entity_sentence_list)
    print("============================\n Dataframe Created \n============================\n")
    
    if export_flag is True:
        df_Test.to_excel(output_file)
        print("File Exported!!!!!")
    else:
        print("處理完成，不產生檔案")
        
    return df_Test,word_sentence_list,pos_sentence_list,entity_sentence_list

In [20]:
"""
108.12.10 還有pop到原始檔的問題待修

"""

def gen_df(in_list):
    df_class_list = list()
    df_ner_list = list()
    
    proc_list = copy.copy(in_list)
    
    count = 0
    for i in range(len(proc_list)):
        for j in range(len(proc_list[i])):
            count = count +1
            a = proc_list[i].pop()
            df_class_list.append(a[2])
            df_ner_list.append(a[3])
        #print(type(new_list[j]),(new_list[j]))
    #print(count)
    extract_df = pd.DataFrame({'類別':df_class_list,'專有名詞':df_ner_list}).copy()
    dd_extract_df = extract_df.drop_duplicates().copy()
    #del extract_df
    
    #===============================
    # 108.12.10 拿掉類別挑選，改成全取
    """
    df_exp = dd_extract_df.loc[dd_extract_df['類別'].isin(['LOC','PERSON', 'ORG', 'LAW', 'EVENT','GPE','FAC'])].copy()
    del dd_extract_df
    df_exp_2 = df_exp[df_exp['專有名詞'].map(len) >= 2].copy()
    del df_exp
    """
    #===============================
    
    #df_exp = dd_extract_df.loc[dd_extract_df['類別'].isin(['LOC','PERSON', 'ORG', 'LAW', 'EVENT','GPE'])].copy()
    #del dd_extract_df
    df_exp_2 = dd_extract_df[dd_extract_df['專有名詞'].map(len) >= 2].copy()
    #del dd_extract_df
    
    #===============================
    # 108.12.10 新增產檔，供後續使用
    
    output_df_file = "NER_reult.xls"
    df_exp_2.to_excel(output_df_file)
    print("產生NER檔:{}".format(output_df_file))
    
    #==============================
    return df_exp_2

# 2. 讀檔

In [6]:
data_path = r"D:\python_kcc\NER\剪報系統匯出-正確\all-new(11554).xlsx"
full_data = pd.read_excel(data_path)
full_data.columns

Index(['序號', '日期', '報別', '內容'], dtype='object')

In [7]:
df_columns = full_data['內容'] #這邊取出就變成Series
len(df_columns),type(df_columns)

(11554, pandas.core.series.Series)

In [23]:
#ls_Test = list(text[200:250])
#df_Test = pd.DataFrame(ls_Test,columns=["Text"])

# 抽樣

In [65]:
#sample_columns = df_columns.sample(n=100).copy()
sample_columns[:5]

11434    直球對決！高雄市長選情白熱化，民進黨籍高雄市長候選人陳其邁與國民黨籍高雄市長候選人韓國瑜終於...
10283    伊甸基金會在高市鳳山區成立「溫馨家園」昨開幕，提供十六到六十四歲身心障礙者日間照顧服務，訓練...
8921     台北市政府年底不再與救國團劍潭青年活動中心續約，包括高雄市及新北市也有土地租給救國團的青年活...
7889     唐晞宇（高雄市，經理） 政府不應該當鴕鳥，公布液化土壤區域是好事，我支持這種作法，讓民眾可瞭...
710      　最近是全台各企業、機關團體舉行年度尾牙旺季，高雄市各飯店、餐廳都接到尾牙訂單，但不少飯店、...
Name: 內容, dtype: object

## 先不用再抽樣，直接讀取 ( for 1081211 present)

## 將抽樣資料匯出成EXCEL檔，供後續利用

In [9]:
output_file_name = "df_sample_108121002"
#sample_columns.to_excel(r"D:\python_kcc\NER\NER_export\\" + output_file_name + ".xls",header=['內容'])

## 從EXCEL讀取抽樣資料

In [10]:
data_path = (r"D:\python_kcc\NER\NER_export\\" + output_file_name + ".xls")
in_data = pd.read_excel(data_path,index_col= 0)

In [11]:
type(in_data),len(in_data),in_data[:5]

(pandas.core.frame.DataFrame,
 100,
                                                       內容  專有名詞  標註人
 11434  直球對決！高雄市長選情白熱化，民進黨籍高雄市長候選人陳其邁與國民黨籍高雄市長候選人韓國瑜終於...   NaN  NaN
 10283  伊甸基金會在高市鳳山區成立「溫馨家園」昨開幕，提供十六到六十四歲身心障礙者日間照顧服務，訓練...   NaN  NaN
 8921   台北市政府年底不再與救國團劍潭青年活動中心續約，包括高雄市及新北市也有土地租給救國團的青年活...   NaN  NaN
 7889   唐晞宇（高雄市，經理） 政府不應該當鴕鳥，公布液化土壤區域是好事，我支持這種作法，讓民眾可瞭...   NaN  NaN
 710    　最近是全台各企業、機關團體舉行年度尾牙旺季，高雄市各飯店、餐廳都接到尾牙訂單，但不少飯店、...   NaN  NaN)

## 進行CKIPTAGGER，取出完整DF及3個LIST

In [12]:
demo_df,ws_list,pos_list,ner_list = word_proc_path_all(data_path = data_path,
          column_name = "內容",
          #export_flag = True,output_file = "sample_NER_rlt_108121002.xls")
           export_flag = False,output_file = "sample_NER_rlt_108121002.xls")                                           

輸入檔案格式為 .xls
取得list，長度為：100
 載入 WS 資料...
Got list
 載入 WS 資料...
 開始執行 WS

 WS 完成
載入 POS 資料...
 開始執行 POS 

 POS 完成
載入NER 資料...
 開始執行 NER 

 NER 完成

 Dataframe Created 

處理完成，不產生檔案


In [14]:
df = pd.read_excel("sample_NER_rlt_1081210.xls",index_col=0)
df.columns,df[:1]

(Index(['內容', 'WS', 'POS', 'NER'], dtype='object'),
                                                   內容  \
 0  直球對決！高雄市長選情白熱化，民進黨籍高雄市長候選人陳其邁與國民黨籍高雄市長候選人韓國瑜終於...   
 
                                                   WS  \
 0  ['直', '球', '對決', '！', '高雄', '市長', '選情', '白熱化',...   
 
                                                  POS  \
 0  ['VH', 'Na', 'VA', 'EXCLAMATIONCATEGORY', 'Nc'...   
 
                                                  NER  
 0  {(82, 83, 'CARDINAL', '2'), (154, 157, 'DATE',...  )

In [21]:
new_ner_list = copy.copy(ner_list)

In [28]:
#ner_list[:5]

# 產出NER DataFrame及excel

In [25]:
df_ner_dict = gen_df(new_ner_list)
df_ner_dict[:5]

產生NER檔:NER_reult.xls


,類別,專有名詞
0,PERSON,韓國瑜
1,ORG,民進黨籍
2,TIME,黃昏
3,ORG,國民黨籍
6,TIME,昨午


# ===============================
# 測試：結巴斷詞比較
# ===============================

# 結巴斷詞─原始字典

In [29]:
from io import StringIO
import io
import os.path
import glob
import pandas as pd
import csv
import numpy as np
from matplotlib import pyplot
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
#from keras.utils import to_categorical

In [30]:
import jieba
from jieba import analyse
import string
#from gensim.models.word2vec import Word2Vec
#import word2vec
import logging
from gensim.models import word2vec
#https://github.com/danielfrg/word2vec/blob/master/examples/word2vec.ipynb
#https://anaconda.org/anaconda/word2vec

C:\Users\b1194\AppData\Local\Continuum\anaconda3\envs\Data_Science\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [31]:
# 載入停止詞詞庫
stopword_set = set()
path_stop_word = r"D:\python_kcc\dict\stop_word.txt"
with open(path_stop_word,'r', encoding='utf-8') as stopwords:
    for stopword in stopwords:
        stopword_set.add(stopword.strip('\n'))
        
type(stopword_set),len(stopword_set)

(set, 750)

# 取出抽樣資料

In [32]:
sample_data = in_data["內容"]
sample_data.head(5)

11434    直球對決！高雄市長選情白熱化，民進黨籍高雄市長候選人陳其邁與國民黨籍高雄市長候選人韓國瑜終於...
10283    伊甸基金會在高市鳳山區成立「溫馨家園」昨開幕，提供十六到六十四歲身心障礙者日間照顧服務，訓練...
8921     台北市政府年底不再與救國團劍潭青年活動中心續約，包括高雄市及新北市也有土地租給救國團的青年活...
7889     唐晞宇（高雄市，經理） 政府不應該當鴕鳥，公布液化土壤區域是好事，我支持這種作法，讓民眾可瞭...
710      　最近是全台各企業、機關團體舉行年度尾牙旺季，高雄市各飯店、餐廳都接到尾牙訂單，但不少飯店、...
Name: 內容, dtype: object

In [33]:
strCut = jieba.cut(sample_data)

In [36]:
path_b_dict = r"D:\python_kcc\dict\dict.big.txt"
path_CIS_dict = r"D:\python_kcc\dict\CIS_Dict.txt"
jieba.set_dictionary(path_b_dict)
jieba.load_userdict(path_CIS_dict)

jieba_ori_cut_result = list()
count = 0
for ele in sample_data:
    count = count +1 
    #print(type(ele))
    str_o = jieba.cut(ele)
    str_list = [word for word in str_o if word not in stopword_set]
    strout = " ".join(str_list)
    
    jieba_ori_cut_result.append(strout)
    #cut_result.append(pd.Series(str_list))
    #a = pd.Series(jieba.cut(ele))
    #print(a)
    #cut_result.append(a)
print(len(jieba_ori_cut_result))
print(count)

Building prefix dict from D:\python_kcc\dict\dict.big.txt ...
I1211 08:37:22.138199 14568 __init__.py:111] Building prefix dict from D:\python_kcc\dict\dict.big.txt ...
Loading model from cache C:\Users\b1194\AppData\Local\Temp\jieba.u9a1c961e2028b38db4173ee3c9b16c71.cache
I1211 08:37:22.140194 14568 __init__.py:131] Loading model from cache C:\Users\b1194\AppData\Local\Temp\jieba.u9a1c961e2028b38db4173ee3c9b16c71.cache
Loading model cost 0.969 seconds.
I1211 08:37:23.108602 14568 __init__.py:163] Loading model cost 0.969 seconds.
Prefix dict has been built succesfully.
I1211 08:37:23.109600 14568 __init__.py:164] Prefix dict has been built succesfully.


100
100


In [40]:
len(jieba_ori_cut_result),jieba_ori_cut_result[-2:]

(100,
 ['課綱 爭議 陷入 僵持 國民 黨籍 高雄市 議員 黃柏霖 日 召開 記者會 邀 教師 家長 代表 共同 呼籲 教育部 成立 獨立 國家 課程 發展 委員會 教育 回歸 專業 開放 各界 監督 黃柏霖 說 將來 政黨 輪替 就換 課綱 社會 永無寧日   請 政治 退出 教育 高雄市 教師職業 公會 副理事長 陳建 志說 教育 應 保持中立 課綱 微調 惹 爭議 藍綠 教育 棋子 其實 2009 年 教育部 中小學 永續 發展 委員會 提出 國家 課程 發展 委員會 構想 各有 政治 盤算 沒 理會   建議 立法院 制訂 相關 條例 國家 課程 發展 委員會 成為 獨立 法人 官方 代表 大於 / 並應 擴大 各界 參與 未來 每次 政黨 輪替 課綱 改來改去 教育 豈不是 擺來擺去   高雄市 各級 學校 家長 協會 理事長 黃 國盛 則力 挺 反 課綱 學生 說 站 家長 立場 看到 國家 政策 竟 導致 學生 站 抗議 覺得 錯愕 新 課綱 只用 年 12 年 國教 上路 會 廢除 教育部 應該 審慎 評估 暫緩   前鎮高中 歷史 老師 洪 碧霞 則說 朝野 協商 要求 重新 召開 聽審 會 等於 暫緩 新 課綱 回到 原點 重新 審查 覺得 教育部 釋出 善意 基層 老師 後續 授課 會將 新舊 課綱 差異 說明 清楚 請 同學們 相信 老師 回到 學校 地方 堅持 獨立思考',
  '\u3000 國民黨 二 ０ 一六準 總統 參選人 洪秀 柱 昨天 展開 高雄 第二天 拜訪 行程 上午 前往 國民黨 鐵票 區 左營 果貿   社區 造勢 時 眷村 居民 熱情歡迎 洪秀 柱 走訪 果貿 社區 菜市場 攤商 送 鳳梨 粽子 祝 好運 旺旺 總統大選 順利   選 令 十分 感動   \u3000 洪秀 柱 昨日 身穿 桃紅色 襯衫 再度 造訪 高雄 最大 眷村 果貿 社區 住戶 手持 挺柱 必安柱 滅文 標語 活動中心 前   候 洪秀 柱 到訪 洪秀 柱 座車 抵達 現場 時 柱柱姐 加油 總統 聲音 不斷   \u3000 現場 媒體 欲 詢問 洪秀 柱 高雄 地區 流傳 一份 卡柱 連署 書 相關 議題 洪秀 柱並 回應   \u3000 洪秀 柱 表示 上次 來到 果貿 參選 連署 過程 中 長輩 先進 訴說 個人 領表 參選 理

# 取出NER結果作為字典，因為結巴的字典檔為txt

## 從excel檔讀出NER Dataframe
## 取出專有名詞欄位
## 寫入txt檔，供結巴使用

In [41]:
ner_dict = pd.read_excel("NER_reult.xls",index_col=0)
ner_dict[:5]

,類別,專有名詞
0,PERSON,韓國瑜
1,ORG,民進黨籍
2,TIME,黃昏
3,ORG,國民黨籍
6,TIME,昨午


In [42]:
ner_dict = ner_dict['專有名詞']
ner_dict[:5]

0     韓國瑜
1    民進黨籍
2      黃昏
3    國民黨籍
6      昨午
Name: 專有名詞, dtype: object

In [43]:
ner_file = open("NER_dict.txt", "w",encoding = 'UTF-8')

In [44]:
for ner in ner_dict:
    ner_str = str(ner)
    ner_file.write(ner+"\n")

# 引入NER DICT 作為userdict

In [46]:
path_b_dict = r"D:\python_kcc\dict\dict.big.txt"
#path_b_dict =  r"D:\python_kcc\NER\NER_dict.txt"
#path_CIS_dict = r"D:\python_kcc\dict\CIS_Dict.txt"
path_CIS_dict = r"D:\python_kcc\NER\NER_dict.txt"

jieba.set_dictionary(path_b_dict)
jieba.load_userdict(path_CIS_dict)

jieba_cut_result_by_ner = list()
#cut_result_by_ner = list()
count = 0
for ele in sample_data:
    count = count +1 
    #print(type(ele))
    str_o = jieba.cut(ele)
    str_list = [word for word in str_o if word not in stopword_set]
    strout = " ".join(str_list)
    
    jieba_cut_result_by_ner.append(strout)
    #cut_result.append(pd.Series(str_list))
    #a = pd.Series(jieba.cut(ele))
    #print(a)
    #cut_result.append(a)
print(len(jieba_cut_result_by_ner))
print(count)

Building prefix dict from D:\python_kcc\dict\dict.big.txt ...
I1211 08:39:20.953428 14568 __init__.py:111] Building prefix dict from D:\python_kcc\dict\dict.big.txt ...
Loading model from cache C:\Users\b1194\AppData\Local\Temp\jieba.u9a1c961e2028b38db4173ee3c9b16c71.cache
I1211 08:39:20.955400 14568 __init__.py:131] Loading model from cache C:\Users\b1194\AppData\Local\Temp\jieba.u9a1c961e2028b38db4173ee3c9b16c71.cache
Loading model cost 0.962 seconds.
I1211 08:39:21.917177 14568 __init__.py:163] Loading model cost 0.962 seconds.
Prefix dict has been built succesfully.
I1211 08:39:21.918188 14568 __init__.py:164] Prefix dict has been built succesfully.


100
100


In [135]:
#sample_data

In [149]:
#ner_dict_CKIP = read_ner_dict.to_dict('records')

# 產生CKIPTAGGER 用的字典檔

In [47]:
# 用毒CSV的方式讀取前面匯出的txt
read_ner_dict = pd.read_csv(path_CIS_dict, sep="\n", header=None)
# 加上Column name
read_ner_dict.columns = ['NER']
read_ner_dict[:3],type(read_ner_dict)

(    NER
 0   韓國瑜
 1  民進黨籍
 2    黃昏, pandas.core.frame.DataFrame)

In [48]:
list_ner_dict = list(read_ner_dict['NER'])
list_ner_dict[:5]

['韓國瑜', '民進黨籍', '黃昏', '國民黨籍', '昨午']

In [49]:
from ckiptagger import construct_dictionary
dict_for_CKIP = dict((el,1) for el in list_ner_dict)
dict_for_CKIP = construct_dictionary(dict_for_CKIP)

# 引入NER 字典做CKIP 斷詞

In [50]:
from ckiptagger import WS
ws = WS("./data")

In [51]:
ws_list_with_ori_dict_encouraged = ws(sample_data,
                                 #sentence_segmentation=True, # To consider delimiters
                                 #segment_delimiter_set = {",", "。", ":", "?", "!", ";"},
                           #segment_delimiter_set = stopword_set,
                                # This is the defualt set of delimiters
                                #recommend_dictionary = dict_for_CKIP, # words in this dictionary are encouraged
                                 #coerce_dictionary = dict_for_CKIP, # words in this dictionary are forced
                            )

In [52]:
#encouraged
ws_list_with_ner_dict_encouraged = ws(sample_data,
                                 sentence_segmentation=True, # To consider delimiters
                                 segment_delimiter_set = {",", "。", ":", "?", "!", ";"},
                           #segment_delimiter_set = stopword_set,
                                # This is the defualt set of delimiters
                                recommend_dictionary = dict_for_CKIP, # words in this dictionary are encouraged
                                 #coerce_dictionary = dict_for_CKIP, # words in this dictionary are forced
                            )

In [53]:
ws_list_with_ner_dict_forced = ws(sample_data,
                                 sentence_segmentation=True, # To consider delimiters
                                 segment_delimiter_set = {",", "。", ":", "?", "!", ";"},
                           #segment_delimiter_set = stopword_set,
                                # This is the defualt set of delimiters
                                #recommend_dictionary = dict_for_CKIP, # words in this dictionary are encouraged
                                 coerce_dictionary = dict_for_CKIP, # words in this dictionary are forced
                            )

In [54]:
for i in range(len(cut_result)):
    print("==================  第{}列 ==============".format(i))
    print("結巴-原始字典:\n{}\n".format(jieba_ori_cut_result[i]))
    print("結巴-NER字典:\n{}\n".format(jieba_cut_result_by_ner[i]))
    print("CKIP-未引入NER辭典:\n{}\n".format(ws_list_with_ori_dict_encouraged[i]))
    print("CKIP-NER字典-encouraged:\n{}\n".format(ws_list_with_ner_dict_encouraged[i]))
    print("CKIP-NER字典-forced:\n{}\n".format(ws_list_with_ner_dict_forced[i]))
    #continue

==================  第0列 ==============
結巴-原始字典:
直球 對決 高雄市 長 選情 白熱化 民進黨 籍 高雄市 長 候選人 陳 其邁 國民 黨籍 高雄市 長 候選人 韓國 瑜 終於 正面 對決 昨 藍綠 陣營 達成 共識 將於 本月 19 日 舉辦 電視 政見 辯論 韓陳 參加 採一 直球 對決 式 辯論 方式   陳 其邁 昨 傍晚 黃昏市場 拜票 前 受訪 表示 昨午 立委 趙天麟 代表 韓國 瑜 陣營 代表 達成 共識 韓國 瑜將 19 日 共同 參加 一場 民間 舉辦 公開 辯論會 採一 方式 問韓 韓問 辯論

結巴-NER字典:
直球 對決 高雄 市長 選情 白熱化 民進黨籍 高雄 市長 候選人 陳其邁 國民黨籍 高雄 市長 候選人 韓國瑜 終於 正面 對決 昨 藍綠 陣營 達成 共識 將於 本月19日 舉辦 電視 政見 辯論 韓陳 參加 採一 直球 對決 式 辯論 方式   陳其邁 昨傍晚 黃昏 市場 拜票 前 受訪 表示 昨午 立委 趙天麟 代表 韓國瑜 陣營 代表 達成 共識 韓國瑜 將在 19日 共同 參加 一場 民間 舉辦 公開 辯論會 採一 方式 問韓 韓問 辯論

CKIP-未引入NER辭典:
['直', '球', '對決', '！', '高雄', '市長', '選情', '白熱化', '，', '民進黨籍', '高雄', '市長', '候選人', '陳其邁', '與', '國民黨籍', '高雄', '市長', '候選人', '韓國瑜', '終於', '要', '正面', '對決', '，', '昨', '藍', '綠', '陣營', '達成', '共識', '，', '將', '於', '本', '月', '19日', '舉辦', '電視', '政見', '辯論', '，', '只有', '韓陳', '2', '人', '參加', '，', '採', '一', '對', '一', '、', '「', '直球', '對決式', '辯論', '」', '方式', '。', ' ', '陳其邁', '昨', '傍晚', '到', '黃昏', '市場', '拜票', '前', '受', '訪', '表示', '，', '昨午', '立委', '趙天麟', '代表', '他', '與', '韓國瑜', '陣

# 取出特定類別的NER作字典

In [55]:
ner_dict = pd.read_excel("NER_reult.xls",index_col=0)
#ner_dict[:5]
#print(ner_dict.describe())

print("全部類別:")
print(ner_dict['類別'].unique())

#ner_dict[ner_dict['類別'] == 'WORK_OF_ART'] #印出特定類別內容

#ner_dict.groupby('類別').count()

全部類別:
['PERSON' 'ORG' 'TIME' 'GPE' 'DATE' 'CARDINAL' 'ORDINAL' 'LOC' 'FAC'
 'QUANTITY' 'MONEY' 'PERCENT' 'NORP' 'PRODUCT' 'EVENT' 'LAW' 'WORK_OF_ART'
 'LANGUAGE']


In [56]:
#查特定詞屬於什麼類別
ner_dict.loc[ner_dict['專有名詞'] == '亞洲有機樂活產業展']
ner_dict.loc[ner_dict['專有名詞'] == '農業展']
#ner_dict.where(ner_dict['專有名詞'] == '亞洲有機樂活產業展')

,類別,專有名詞


In [57]:
#取出特定類別
ner_dict_sel_class = ner_dict.loc[ner_dict['類別'].isin(['LOC','DATE','EVENT','FAC','GPE','LAW','LOC','ORG','PERSON', 'TIME', 'WORK_OF_ART'])]
ner_dict_sel_class = ner_dict['專有名詞']
ner_file_sel_class = open("NER_dict_sel_class.txt", "w",encoding = 'UTF-8')

In [58]:
for ner in ner_dict_sel_class:
    ner_str = str(ner)
    ner_file_sel_class.write(ner+"\n")

In [313]:
# 用毒CSV的方式讀取前面匯出的txt
path_ner_dict = r"D:\python_kcc\NER\NER_dict_sel_class.txt"
read_ner_dict = pd.read_csv(path_CIS_dict, sep="\n", header=None)
# 加上Column name
read_ner_dict.columns = ['NER']
read_ner_dict = read_ner_dict.dropna()
read_ner_dict[:5],type(read_ner_dict)

(      NER
 1     黃馨慧
 2    第六分局
 3  六百六十八元
 4     李如芳
 5     蔡義猛, pandas.core.frame.DataFrame)

In [314]:
list_ner_dict = list(read_ner_dict['NER'])
list_ner_dict[:5]

['黃馨慧', '第六分局', '六百六十八元', '李如芳', '蔡義猛']

In [316]:
from ckiptagger import construct_dictionary
dict_for_CKIP_sel_class = dict((el,1) for el in list_ner_dict)
dict_for_CKIP_sel_class = construct_dictionary(dict_for_CKIP_sel_class)

In [318]:
ws_list_with_ner_dict_encouraged_sel_class = ws(sample_data,
                                 sentence_segmentation=True, # To consider delimiters
                                 segment_delimiter_set = {",", "。", ":", "?", "!", ";"},
                           #segment_delimiter_set = stopword_set,
                                # This is the defualt set of delimiters
                                recommend_dictionary = dict_for_CKIP_sel_class, # words in this dictionary are encouraged
                                 #coerce_dictionary = dict_for_CKIP, # words in this dictionary are forced
                            )

In [321]:
for i in range(len(25)):
    print("==================  第{}列 ==============".format(i))
    #print("結巴-原始字典:\n{}\n".format(cut_result[i]))
    #print("結巴-NER字典:\n{}\n".format(cut_result_by_ner[i]))
    print("CKIP-NER字典-encouraged:\n{}\n".format(ws_list_with_ner_dict_encouraged[i]))
    print("CKIP-NER字典-forced:\n{}\n".format(ws_list_with_ner_dict_forced[i]))
    print("CKIP-NER字典-encouraged，特定類別:\n{}\n".format(ws_list_with_ner_dict_encouraged_sel_class[i]))
    #continue

==================  第0列 ==============
CKIP-NER字典-encouraged:
['直', '球', '對決', '！', '高雄', '市長', '選情', '白熱化', '，', '民進黨籍', '高雄', '市長', '候選人', '陳其邁', '與', '國民黨籍', '高雄', '市長', '候選人', '韓國瑜', '終於', '要', '正面', '對決', '，', '昨', '藍', '綠', '陣營', '達成', '共識', '，', '將', '於', '本月19日', '舉辦', '電視', '政見', '辯論', '，', '只有', '韓', '陳', '2', '人', '參加', '，', '採', '一', '對', '一', '、', '「', '直球', '對決式', '辯論', '」', '方式', '。', ' ', '陳其邁', '昨傍晚', '到', '黃昏', '市場', '拜票', '前', '受', '訪', '表示', '，', '昨午', '立委', '趙天麟', '代表', '他', '與', '韓國瑜', '陣營', '代表', '達成', '共識', '，', '他', '和', '韓國瑜', '將', '在', '19日', '共同', '參加', '一', '場', '由', '民間', '舉辦', '公開', '辯論會', '，', '採', '一', '對', '一', '方式', '，', '就', '是', '「', '我', '問', '韓', '、', '韓', '問', '我', '」', '的', '辯論', '。']

CKIP-NER字典-forced:
['直', '球', '對決', '！', '高雄市', '長', '選情', '白熱化', '，', '民進黨籍', '高雄市', '長', '候選人', '陳其邁', '與', '國民黨籍', '高雄市', '長', '候選人', '韓國瑜', '終於', '要', '正面', '對決', '，', '昨', '藍', '綠', '陣營', '達成', '共識', '，', '將', '於', '本月19日', '舉辦', '電視', '政見', '辯論', '，', '只有', 